In [1]:
import numbers

In [7]:
class IntegerFields:
    def __init__(self,min_,max_):
        self._min = min_
        self._max = max_
        
    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
        
    def __set__(self,instance,value):
        if not isinstance(value,numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer')
        if value < self._min:
            raise ValueError(f'{self.prop_name} must be >={self._min}')
        if value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
    
        instance.__dict__[self.prop_name] = value
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name,None)

In [8]:
class Person:
    age = IntegerFields(0,100)

In [9]:
p = Person()

In [10]:
p.age = 5

In [11]:
p.age

5

In [12]:
try:
    p.age = 200
except ValueError as ex:
    print(ex)

age must be <= 100


In [14]:
import unittest

In [21]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity = 2)
    result = runner.run(suite)

In [22]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerFields(0,10)
    
    def test_set_age_ok(self):
        p = self.Person()
        p.age = 0
        self.assertEqual(0,p.age)

In [23]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [34]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerFields(0,10)
        
    def test_set_age_ok(self):
        min_ =5
        max_ = 10
        self.Person.age = IntegerFields(5,10)
        
        p = self.Person()
        p.age = 5
        self.assertEqual(5,p.age)

In [35]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ERROR

ERROR: test_set_age_ok (__main__.TestIntegerField)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-34-ce42a2ac4e09>", line 11, in test_set_age_ok
    p.age = 5
  File "<ipython-input-7-af8260a7e332>", line 17, in __set__
    instance.__dict__[self.prop_name] = value
AttributeError: 'IntegerFields' object has no attribute 'prop_name'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


In [37]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerFields(0,10)
        
    def create_person(self,min_,max_):
        self.Person.age = IntegerFields(min_,max_)
        self.Person.age.__set_name__(Person,'age')
        return self.Person()
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_person(min_,max_)
        
        
        p.age = 5
        self.assertEqual(5,p.age)

In [39]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [40]:
class Person:
    a =10

In [41]:
type(Person)

type

In [43]:
type('Person', (),{'a':10})

__main__.Person

In [44]:
type(Person)

type

In [45]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'a': 10,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [46]:
Person.a

10

In [47]:
Person.a =100

In [48]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'a': 100,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [49]:
p = Person()

In [50]:
p.a

100

In [51]:
p.a = 1

In [52]:
p.a

1

In [53]:
p.__dict__

{'a': 1}

In [61]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_,max_):
        obj = type('TestCase',(),{'age':IntegerFields(min_,max_)})
        return obj()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_test_class(min_,max_)
        
        
        p.age = 5
        self.assertEqual(5,p.age)

In [62]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [71]:
   run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [76]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_,max_):
        obj = type('TestCase',(),{'age':IntegerFields(min_,max_)})
        return obj()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_,max_)
        valid_values = range(min_,max_ +1)
        
        for i,value in enumerate(valid_values):
            with self.subTest(test_number = i):
                obj.age = value
                self.assertEqual(value,obj.age)
    
    def test_set_age_invalid(self):
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_,max_)
        
        bad_values = list(range(min_ -5 ,min_))
        bad_values += list(range(max_ +1,max_ +5))
        bad_values += [(10.5,1+0j,'abc',(1,2))]
        
        for i,value in enumerate(bad_values):
            with self.subTest(test_number = i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        obj = self.create_test_class(0,0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age,IntegerFields)
        
        
    def test_set_age_min_only(self):
        min_ = 0
        max_ = None
        obj = self.create_test_class(min_,max_)
        values = range(min_,min_ +100, 10)
        for i,value in enumerate(values):
            with self.subTest(test_number = i):
                obj.age = value
                self.assertEqual(value,obj.age)
                
    def test_set_age_max_only(self):
        min_ =None
        max_= 10
        obj = self.create_test_class(min_,max_)
        values = range(max_ -100, max_, 10)
        for i,value in enumerate(values):
            with self.subTest(test_number = i):
                obj.age = value
                self.assertEqual(value,obj,age)

In [80]:
class IntegerFields:
    def __init__(self,min_ = None,max_ = None):
        self._min = min_
        self._max = max_
        
    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
        
    def __set__(self,instance,value):
        if not isinstance(value,numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.prop_name} must be >={self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
    
        instance.__dict__[self.prop_name] = value
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name,None)

In [81]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_,max_):
        obj = type('TestCase',(),{'age':IntegerFields(min_,max_)})
        return obj()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_,max_)
        valid_values = range(min_,max_ +1)
        
        for i,value in enumerate(valid_values):
            with self.subTest(test_number = i):
                obj.age = value
                self.assertEqual(value,obj.age)
    
    def test_set_age_invalid(self):
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_,max_)
        
        bad_values = list(range(min_ -5 ,min_))
        bad_values += list(range(max_ +1,max_ +5))
        bad_values += [(10.5,1+0j,'abc',(1,2))]
        
        for i,value in enumerate(bad_values):
            with self.subTest(test_number = i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        obj = self.create_test_class(0,0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age,IntegerFields)

In [82]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [83]:
class CharFields:
    def __init__(self,min_ = None,max_ = None):
        min_ = min_ or 0
        min_ = max(0,min_)
        self._min = min_
        self._max = max_
        
    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name
        
    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise ValueError(f'{self.prop_name} must be an string')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >={self._min} chars')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')
    
        instance.__dict__[self.prop_name] = value
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name,None)

In [84]:
class Person:
    name = CharFields(1,10)
    

In [86]:
p = Person()
try:
    p.name = 'Python Rocks'
except ValueError as ex:
    print(ex)

name must be <= 10 chars


In [87]:
p.name = 'John'

In [88]:
p.name 

'John'

In [90]:
class Person:
    name = CharFields(1)

In [91]:
p.name = "I'm a lumberjack and I'm ok"

ValueError: name must be <= 10 chars

In [92]:
p.name = 'a' *1000

ValueError: name must be <= 10 chars

In [97]:
class TestCharField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_,max_):
        obj = type('TestCase',(),{'age':CharFields(min_,max_)})
        return obj()
    
    def test_set_name_ok(self):
        min_ = 1
        max_ = 10
        obj = self.create_test_class(min_,max_+1)
        valid_lengths = range(0,100,10)
        for i,length in enumerate(valid_lengths):
            value = 'a' *length
            with self.subTest(test_number = i):
                obj.name = value
                self.assertEqual(value,obj.name)
            

In [98]:
run_tests(TestCharField)

test_set_name_ok (__main__.TestCharField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [100]:
class BaseValidator:
    def __init__(self,min_ = None,max_ = None):
        self._min = min_
        self._max = max_
        
    def __set_name__(self,owner_class,prop_name):
        self.prop_name = prop_name

    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name,None)
        
        
    def validate(self,value):
        pass
        
    
    def __set__(self,instance,value):
        self.validate(value)
        instance.__dict__[self.prop_name] = value
        
        
    


In [101]:
class Person:
    name = BaseValidator()

In [102]:
p = Person()
p.name = 'Sam'
p.name = ['a','b']

In [103]:
p.name

['a', 'b']

In [104]:
class IntegerValidator(BaseValidator):
    def validate(self,value):
        if not isinstance(value,numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.prop_name} must be >={self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
        

In [105]:
class CharField(BaseValidator):
    def __init__(self,min_,max_):
        min_=max(min_ or 0, 0)
        super().__init__(min_,max_)
        
    def validate(self,value):
        if not isinstance(value,str):
            raise ValueError(f'{self.prop_name} must be an string')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >={self._min} chars')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')

In [107]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField) ... ok
test_set_age_invalid (__main__.TestIntegerField) ... ok
test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK
